---


STOCHASTIC FORCING of (balanced) TURBULENCE
===

In French: Forçage Stochastique de la Turbulence (FSTURB)

This configuration aims at generating slow/balanced turbulence with a stochatic forcing generated in horizontal wavenumber space.
Three dimension forcing fields are generated every $\Delta t$ (*fst_dt* in code) seconds and then linearly interpolated at every time step:

$$
\begin{align}
F(i,j,k,t) &= \frac{(n+1)\Delta t-t}{\Delta t} F_{n\Delta t}(i,j,k) 
+ \frac{t-n\Delta t}{\Delta t} F_{(n+1)\Delta t}(i,j,k) 
\\
\mathrm{for}\;\; n\Delta t&<t<(n+1)\Delta t
\end{align}
$$


The spatial structure of forcing fields are given by:

$$
\begin{align}
F_{n\Delta t}(i,j,k) = 
F_{T} \; 
W(i,j) \;
\times
iFFT
\Big \{
\sum_{m} N_{n\Delta t}(kx,ky,m) \psi(kx,ky,m) \phi_{m}(k)
\Big \}
\end{align}
$$

$ F_{T} $ is a constant corresponding to density forcing (will be declined into $ F_{u}, F_{v} $ later on ).

$ W(i,j) $ is a window that may tapper the forcing in physical space. 
Current implementation tappers in the y direction:

$$ W(i,j) = e^{-((yr(i,j)-y_{F})/(\Delta y_{F}))^{2}} $$

$N_{n\Delta t}(kx,ky,m)$ is a normal white noise.

$ \psi(kx,ky,m) $ is constant through the course of thenumerical simulation and modulates the forcing amplitude spectrally (horizontal, vertical modes).

$$ 
\psi (kx,ky,m)= \underbrace{a_{m}}_{mode} \underbrace{e^{-(kx^{2} + ky^{2} - kF^{2})/(\Delta kF^{2}) }}_{horizontal}
$$

$\phi_{n}$ are vertical normal modes


Implementation
===

- cppdefs.h : *FSTURB* is the cpp key that turns on the forcing
- ana_fsturb.F : new module which regroup initialisation and forcing computations
- main.F : forcing initialisation ( *call init_fsturb*)
- step.F : compute new forcing every $\Delta t$ time steps
- step3d_t.F : new forcing interpolated in time and z
 
Parameters names in code
===

- *fst_Nmode* : Number of modes
- *fst_dt* = 1.*86400
- *fst_a* : $a_{n}$ amplitude des différents modes
- *fst_xmid* : (not implemented)
- *fst_xwid* :
- *fst_ymid* : $ y_{F} $
- *fst_ywid* : $ \Delta y_{F} $
- *fst_FT* : $ F_{T} $
- *fst_kF* : $ kF $
- *fst_dkF* : $ \Delta kF $

Variables names
===

All variables related to the FSTURB forcing are declared in the fsturb modul, in the file 
*ana_fsturb.F*

- *fst_F(i,j,k,t)* : $ F_{n\Delta t}(i,j,k) $
- *fst_FT* : $ F_{T} $ défini en parameter dans le module
- *fst_W* : W(i,j)
- *fst_Noise* : $ N_{n\Delta t}(kx,ky,n) $ 
- *fst_a* : $a_{n}$ amplitude des différents modes
- *fst_psi* : $\psi(kx,ky,n)$
- *phir* : modes calculés dans vmodes


---

# balanced turbulence with internal waves configuration

The cpp key is ITBALT

The type of jet (1 to 4) is defined in `cppdefs.h`

Run time options are found in `croco.in`

## commands from start to finish:

Compilation:
```
./jobcomp
```

1 run at 4 km resolution:
```
python chain_datarmor.py fsturb_4km_0a100j 1 03:00:00 4 fs0 0
# and follow instructions
```

10 runs at 4 km resolution:
```
python chain_datarmor.py itbalt_4km_0a1000j 10 03:00:00 4 itb 0
# and follow instructions
```

Restart:
```
python chain_datarmor.py itbalt_4km_1000a1500j 5 03:00:00 4 itb 1
# and follow instructions
```

## outputs:

see [`iodef.xml`](iodef.xml)

### file_his_*.nc

Relevant temporal variable: `time_counter`

Variables are 2D or 3D:

- `v_a` : temporal averaged of variable $v$

- `v_t_cos` : temporal averaged of $v\times\cos(\omega t)$

- `v_t_sin` : temporal averaged of $v\times\cos(\omega t)$

- `v_t_dirac` : instantaneous value of $v$ at the center of the interval

The relevant time interval for this output has size `freq_op` (2d typically) and is outputed every `output_freq` (25d typically).

![his](../JETN/croco_jetn_time.001.png)


### file_ave_*.nc

Relevant temporal variable: `time_counter`

Variables are 2D and averaged over a temporal window of size `output_freq` (2d typically).

- `v` : temporal averaged of variable $v$

- `v_t_cos` : temporal averaged of $v\times\cos(\omega t)$

- `v_t_sin` : temporal averaged of $v\times\cos(\omega t)$

![ave](../JETN/croco_jetn_time.002.png)

### file_surf_*.nc

Relevant temporal variable: `time_instant`

Variables are 2D (surface) and instantaneous every `output_freq` (10 min typically).

### file_inst_*.nc, file_sta1_*.nc, ...

Relevant temporal variable: `time_instant`

Variables are 1D and instantaneous every `output_freq` (30min typically).

### file_swath_*.nc

Not that useful for now.


### nicer treatment of time coordinate

Work is ongoing around xarray in order to implement useful features for this
such as selection alon non-dim coordinates [issue](https://github.com/pydata/xarray/issues/1603)
